In [1]:
import numpy as np
import h5py
import requests
import os.path
import shutil
import matplotlib.pyplot as plt

In [2]:
def slidingWindow(sequence,winSize=8192,step=1):
    """Returns a generator that will iterate through
    the defined chunks of input sequence.  Input sequence
    must be iterable."""
 
    # Verify the inputs
    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")
    if winSize > len(sequence):
        raise Exception("**ERROR** winSize must not be larger than sequence length.")
 
    # Pre-compute number of chunks to emit
    numOfChunks = ((len(sequence)-winSize)/step)+1
 
    # Do the work
    for i in range(0,numOfChunks*step,step):
        yield sequence[i:i+winSize]

In [3]:
def getData(f, noise=0.1):
    df = []
    ruido = []
    for x in f.values():
        dset = f[x.name].items()
        dnoise = f[x.name].items()
        del dset[0:4]
        del ruido[0:4]
        for index, data in enumerate(dset):
            arr = np.zeros(data[1].shape, dtype='float64')
            data[1].read_direct(arr)
            temp = []
            for i in arr:
                temp.append(i[1])
            #norm = ((temp-min(temp))/(max(temp)-min(temp))*2)-1
            norm = map(lambda x: x+(noise*((np.random.rand()*2)-1)), temp/max(temp))
            dnoise[index] = map(lambda x: x, (noise*((np.random.rand(len(temp))*2)-1)))
            dset[index] = norm
         
        df.append(dset)
        ruido.append(dnoise)   
    return df,ruido

In [4]:
urls = ['https://data.black-holes.org/waveforms/data/SXS:BBH:1361/Lev3/rh_FiniteRadii_CodeUnits.h5',
        'https://data.black-holes.org/waveforms/data/SXS:BBH:0112/Lev5/rh_FiniteRadii_CodeUnits.h5',
        'https://data.black-holes.org/waveforms/data/SXS:BBH:0302/Lev5/rh_FiniteRadii_CodeUnits.h5']

path = 'rh_FiniteRadii_CodeUnits'

data = []

for u in range(len(urls)):
    if (not os.path.exists('dados/' + path +''+ u +'.h5')):
        response = requests.get(urls[u], stream=True)
        if response.status_code == 200:
            with open('dados/' + path +''+ u +'.h5', 'wb') as f:
                shutil.copyfileobj(response.raw, f)
    else:
        print('arquivo já existe')

    f = h5py.File('dados/' + path +''+ u +'.h5', 'r')
    df, noise = getData(f)
    f.close()
    tudo = []
    for i in range(len(df)):
        for k in range(len(df[i])):
            dados = list(slidingWindow(df[i][k], step=8192))
            ruidos = list(slidingWindow(noise[i][k], step=8192))
            all = dados + ruidos
            tudo = tudo + all

    data = data + tudo

print(len(data))

TypeError: cannot concatenate 'str' and 'int' objects